### Base meme coins monthly momemtum strategy

In [3]:
# get category symbols
category = "base-meme-coins"
import requests
url = "https://api.coingecko.com/api/v3/coins/markets"
params = {
    "vs_currency": "usd",
    "category": category,
    "order": "market_cap_desc",
    "per_page": 300,
    "page": 1
}

response = requests.get(url, params=params)
data = response.json()

# Extract the symbols
defi_symbols = [coin['symbol'].upper() + '/USDT' for coin in data]
print("DeFi Symbols:", defi_symbols)

DeFi Symbols: ['BRETT/USDT', 'DEGEN/USDT', 'TOSHI/USDT', 'KEYCAT/USDT', 'MIGGLES/USDT', 'HIGHER/USDT', 'BENJI/USDT', 'DOGINME/USDT', 'MFER/USDT', 'PEPE/USDT', 'SKI/USDT', 'TYBG/USDT', 'CHOMP/USDT', 'ELONRWA/USDT', 'MOCHI/USDT', 'TOBY/USDT', 'NORMIE/USDT', 'ANDY/USDT', 'CHAD/USDT', 'BOOMER/USDT', 'LUCKY/USDT', 'PEEZY/USDT', 'FLOPPA/USDT', 'BAMBOO/USDT', 'CRASH/USDT', 'COINYE/USDT', 'SUMI/USDT', 'WEIRDO/USDT', 'SKOP/USDT', 'BRIUN/USDT', 'WEWE/USDT', 'KENDU/USDT', 'AEROBUD/USDT', 'CONDO/USDT', 'AYB/USDT', 'BOOSHI/USDT', 'BEE/USDT', 'TIMI/USDT', 'BASED/USDT', 'ROCK/USDT', 'TN100X/USDT', 'BENI/USDT', 'WOLF/USDT', 'PONCHO/USDT', 'AETHER/USDT', '$WSB/USDT', 'NULL/USDT', 'TAD/USDT', 'WOOF/USDT', 'FLIES/USDT', 'TYLER/USDT', 'WORKIE/USDT', 'CARLO/USDT', 'LAMBOW/USDT', 'MORK/USDT', 'MVP/USDT', 'GIF/USDT', 'WGC/USDT', 'ALF/USDT', 'BOSHI/USDT', 'CORGI/USDT', 'WAGMI/USDT', 'BRIAN/USDT', 'SACA/USDT', 'BERF/USDT', 'BCP/USDT', 'FOMO/USDT', 'ANIME/USDT', 'CAT/USDT', 'TYBENG/USDT', 'MOB/USDT', '$SODA/USD

In [4]:
# Fetch aviable symbols from Bybit
import ccxt

exchange = ccxt.bybit()
exchange.load_markets()
all_symbols = exchange.symbols

# Filter to include only DeFi symbols available on the exchange
available_defi_symbols = [symbol for symbol in defi_symbols if symbol in all_symbols]
print("Available DeFi Symbols:", available_defi_symbols)

Available DeFi Symbols: ['BRETT/USDT', 'DEGEN/USDT', 'MFER/USDT', 'PEPE/USDT', 'CAT/USDT', 'CATS/USDT']


In [6]:
# Fetch ohlcv from Bybit
import numpy as np
import pandas as pd
import datetime as dt
import pytz
import plotly.express as px
import plotly.graph_objects as go

symbols = available_defi_symbols
timeframe = '1d'
limit = 252

# Create an empty list to store DataFrames for each symbol
data_frames = []

for symbol in symbols:
    try:
        ohlcv_data = exchange.fetch_ohlcv(symbol, timeframe, limit=limit)
        df = pd.DataFrame(ohlcv_data, columns=['timestamp', 'open', 'high', 'low', 'close', 'volume'])
        df['symbol'] = symbol

        # Convert timestamp to datetime format for readability
        df['timestamp'] = pd.to_datetime(df['timestamp'], unit='ms')

        data_frames.append(df)
    except Exception as e:
        print(f"Could not fetch data for {symbol}: {e}")

# Concatenate all DataFrames into a single DataFrame
all_data = pd.concat(data_frames, ignore_index=True)
all_data.head()

,timestamp,open,high,low,close,volume,symbol
0,2024-04-18,0.03000,0.06808,0.03000,0.06073,85421673.57,BRETT/USDT
1,2024-04-19,0.06073,0.06506,0.05462,0.06073,73203544.52,BRETT/USDT
2,2024-04-20,0.06073,0.07110,0.05944,0.06989,61902216.04,BRETT/USDT
3,2024-04-21,0.06989,0.07301,0.06401,0.06447,44774385.28,BRETT/USDT
4,2024-04-22,0.06447,0.07100,0.06443,0.06799,39319352.10,BRETT/USDT


In [7]:
# Check min max date
print(all_data['timestamp'].min(), all_data['timestamp'].max())

2024-03-10 00:00:00 2024-11-16 00:00:00


In [8]:
# Filter dublicates and unnecesary timestamps
# Ensure that 'timestamp' is in datetime format
all_data['timestamp'] = pd.to_datetime(all_data['timestamp'])

# Find the latest date in the data
latest_date = all_data['timestamp'].max()

# Calculate the cutoff date for 252 days before the latest date
cutoff_date = latest_date - pd.Timedelta(days=252)

# Filter the data to include only rows within the last 252 days
filtered_data = all_data[all_data['timestamp'] >= cutoff_date]

# Check the new date range to confirm
print("Filtered data range:", filtered_data['timestamp'].min(), "-", filtered_data['timestamp'].max())

# Identify duplicate entries based on 'timestamp' and 'symbol'
duplicates = filtered_data[filtered_data.duplicated(subset=['timestamp', 'symbol'], keep=False)]
print("Duplicate entries:")
duplicates.info()

# Remove duplicate entries, keeping only the first occurrence
filtered_data = filtered_data.drop_duplicates(subset=['timestamp', 'symbol'], keep='first')


Filtered data range: 2024-03-10 00:00:00 - 2024-11-16 00:00:00
Duplicate entries:
<class 'pandas.core.frame.DataFrame'>
Index: 0 entries
Data columns (total 7 columns):
 #   Column     Non-Null Count  Dtype         
---  ------     --------------  -----         
 0   timestamp  0 non-null      datetime64[ns]
 1   open       0 non-null      float64       
 2   high       0 non-null      float64       
 3   low        0 non-null      float64       
 4   close      0 non-null      float64       
 5   volume     0 non-null      float64       
 6   symbol     0 non-null      object        
dtypes: datetime64[ns](1), float64(5), object(1)
memory usage: 0.0+ bytes


In [11]:
# get closed price data
df = filtered_data[['timestamp', 'close', 'symbol']]
df.head()

,timestamp,close,symbol
0,2024-04-18,0.06073,BRETT/USDT
1,2024-04-19,0.06073,BRETT/USDT
2,2024-04-20,0.06989,BRETT/USDT
3,2024-04-21,0.06447,BRETT/USDT
4,2024-04-22,0.06799,BRETT/USDT


In [12]:
# pivot by timestamp and symbol
df_pivoted = df.pivot(index='timestamp', columns='symbol', values='close')

print(df_pivoted.head())
print(df_pivoted.tail())

symbol      BRETT/USDT  CAT/USDT  CATS/USDT  DEGEN/USDT  MFER/USDT  PEPE/USDT
timestamp                                                                    
2024-03-10         NaN  0.001654        NaN         NaN        NaN   0.000009
2024-03-11         NaN  0.001393        NaN         NaN        NaN   0.000009
2024-03-12         NaN  0.001439        NaN         NaN        NaN   0.000008
2024-03-13         NaN  0.001409        NaN         NaN        NaN   0.000009
2024-03-14         NaN  0.001253        NaN         NaN        NaN   0.000010
symbol      BRETT/USDT  CAT/USDT  CATS/USDT  DEGEN/USDT  MFER/USDT  PEPE/USDT
timestamp                                                                    
2024-11-12     0.11912  0.000521   0.000043     0.01965    0.02821   0.000014
2024-11-13     0.13856  0.000458   0.000041     0.01788    0.03092   0.000021
2024-11-14     0.14356  0.000452   0.000045     0.02557    0.03597   0.000021
2024-11-15     0.17797  0.000478   0.000043     0.02630    0.049

In [13]:
# Calculate monthly returns
returns_df = df_pivoted.pct_change().resample("ME").agg(lambda x: (x+1).prod()-1)
# Convert the index of past_cum_return_df to timezone-aware UTC
returns_df.index = returns_df.index.tz_localize('UTC')
returns_df.tail(7)

symbol,BRETT/USDT,CAT/USDT,CATS/USDT,DEGEN/USDT,MFER/USDT,PEPE/USDT
timestamp,,,,,,
2024-05-31 00:00:00+00:00,1.449343,0.058888,0.000000,0.160392,-0.278431,1.311267
2024-06-30 00:00:00+00:00,0.750587,-0.472709,0.000000,-0.648000,-0.404891,-0.226294
2024-07-31 00:00:00+00:00,-0.206399,-0.126953,0.000000,-0.425505,-0.332420,-0.073937
2024-08-31 00:00:00+00:00,-0.357850,-0.011186,0.000000,-0.228571,-0.300274,-0.312302
2024-09-30 00:00:00+00:00,0.173434,0.194570,0.000000,0.225071,0.638319,0.377254
2024-10-31 00:00:00+00:00,-0.036202,-0.212121,-0.776525,1.116279,0.302506,-0.128548
2024-11-30 00:00:00+00:00,0.981274,0.139423,0.222552,1.892308,1.244617,1.409081


In [14]:
# Create a line plot with Plotly
fig = px.line(returns_df, x=returns_df.index, y=returns_df.columns, title="Monthly returns")
# Adjust the layout for a wider plot
fig.update_layout(width=1200, height=600, xaxis_title="Timestamp", yaxis_title="Returns")
# Show the interactive plot
fig.show()

In [15]:
# rolling cumulative for 6 months return
past_cum_return_df = (returns_df + 1).rolling(6).apply(np.prod) - 1

past_cum_return_df.tail(7)

symbol,BRETT/USDT,CAT/USDT,CATS/USDT,DEGEN/USDT,MFER/USDT,PEPE/USDT
timestamp,,,,,,
2024-05-31 00:00:00+00:00,NaN,NaN,NaN,NaN,NaN,NaN
2024-06-30 00:00:00+00:00,NaN,NaN,NaN,NaN,NaN,NaN
2024-07-31 00:00:00+00:00,NaN,NaN,NaN,NaN,NaN,NaN
2024-08-31 00:00:00+00:00,0.314013,-0.732769,0.000000,-0.846658,-0.942431,-0.110609
2024-09-30 00:00:00+00:00,0.541907,-0.720042,0.000000,-0.924442,-0.905684,0.189559
2024-10-31 00:00:00+00:00,1.471249,-0.546347,-0.776525,-0.530686,-0.571961,0.366853
2024-11-30 00:00:00+00:00,0.998994,-0.511843,-0.726790,0.169778,0.331522,0.424698


In [16]:
# Show cumulative return from past 6 months
fig = px.line(past_cum_return_df, x=past_cum_return_df.index, y=past_cum_return_df.columns, title="Rolling cumulative for 6 months return")
fig.update_layout(width=1200, height=600, xaxis_title="Timestamp", yaxis_title="Cumulative Returns")
fig.show()

In [17]:
# Define formation and measurement period
# Define the dates with timezone information
end_of_measurement_period = dt.datetime(2024, 9, 30, tzinfo=pytz.UTC)
formation_period = dt.datetime(2024, 10, 31, tzinfo=pytz.UTC)

end_of_measurement_period_return_df = past_cum_return_df.loc[end_of_measurement_period]
# Transpose the DataFrame so that tickers become rows and returns become a single column
end_of_measurement_period_return_df = end_of_measurement_period_return_df.T
end_of_measurement_period_return_df = end_of_measurement_period_return_df.reset_index()
# Rename columns for clarity
end_of_measurement_period_return_df.columns = ['Symbol', 'Return']
end_of_measurement_period_return_df.head()

,Symbol,Return
0,BRETT/USDT,0.541907
1,CAT/USDT,-0.720042
2,CATS/USDT,0.000000
3,DEGEN/USDT,-0.924442
4,MFER/USDT,-0.905684


In [18]:
# highest momentum in the positive direction
end_of_measurement_period_return_df.loc[end_of_measurement_period_return_df.iloc[:,1].idxmax()]

Symbol    BRETT/USDT
Return      0.541907
Name: 0, dtype: object

In [19]:
# highest momentum in the negative direction
end_of_measurement_period_return_df.loc[end_of_measurement_period_return_df.iloc[:,1].idxmin()]

Symbol    DEGEN/USDT
Return     -0.924442
Name: 3, dtype: object

In [24]:
# rank symbols with quantiles
end_of_measurement_period_return_df['rank'] = pd.qcut(end_of_measurement_period_return_df.iloc[:,1], 3, labels=False)
end_of_measurement_period_return_df.head(7)

,Symbol,Return,rank
0,BRETT/USDT,0.541907,2
1,CAT/USDT,-0.720042,1
2,CATS/USDT,0.000000,1
3,DEGEN/USDT,-0.924442,0
4,MFER/USDT,-0.905684,0
5,PEPE/USDT,0.189559,2


In [25]:
# Create a bar plot to visualize ranks
fig = px.bar(
    end_of_measurement_period_return_df, 
    x='Symbol', 
    y='Return', 
    color='rank',  # Color by rank to visually distinguish groups
    title="Asset Returns and Ranks",
    labels={'rank': 'Rank'},
)

# Customize layout for readability
fig.update_layout(
    xaxis_title="Symbol",
    yaxis_title="Return",
    coloraxis_colorbar=dict(title="Rank"),
    width=1000,
    height=600
)

fig.show()


In [26]:
# select highest rank
long_stocks = end_of_measurement_period_return_df.loc[end_of_measurement_period_return_df["rank"]==2,"Symbol"].values
long_stocks

array(['BRETT/USDT', 'PEPE/USDT'], dtype=object)

In [27]:
# select lowest rank
short_stocks = end_of_measurement_period_return_df.loc[end_of_measurement_period_return_df["rank"]==0,"Symbol"].values
short_stocks

array(['DEGEN/USDT', 'MFER/USDT'], dtype=object)

In [28]:
# long strategy return
from dateutil.relativedelta import relativedelta

long_return_df = returns_df.loc[formation_period + relativedelta(months=1), \
                                   returns_df.columns.isin(long_stocks)]
long_return_df

symbol
BRETT/USDT    0.981274
PEPE/USDT     1.409081
Name: 2024-11-30 00:00:00+00:00, dtype: float64

In [29]:
# short strategy return
short_return_df = returns_df.loc[formation_period + relativedelta(months=1), \
                                   returns_df.columns.isin(short_stocks)]
short_return_df

symbol
DEGEN/USDT    1.892308
MFER/USDT     1.244617
Name: 2024-11-30 00:00:00+00:00, dtype: float64

In [31]:
# Define Strategy tester class
class strategyTester:
    """Calculate metrics based on cumulative return"""

    def __init__(self, df: pd.Series, period: str, riskfree_rate: float = 0.03, initial_wealth: int = 1000):
        self.annualized_return_value = self.annualized_return(df, period)
        self.annualized_volatility_value = self.annualized_volatility(df, period)
        self.sharpe_ratio_value = self.sharpe_ratio(df, riskfree_rate)
        self.max_drawdown_value = self.max_drawdown(df, initial_wealth)
        
    # annualized return
    @staticmethod
    def annualized_return(df: pd.Series, period: str):
        if period == 'daily':
            factor = 252
        elif period == 'weekly':
            factor = 52
        elif period == 'monthly':
            factor = 12
        else:
            raise ValueError("Invalid period. Use 'daily', 'weekly', or 'monthly'.")
        
        annualized_return = (1 + df) ** (factor / len(df)) - 1
        print(f"Annualized return: {annualized_return}")
        return annualized_return

    # annualized volatility
    @staticmethod
    def annualized_volatility(df: pd.Series, period: str):
        if period == 'daily':
            factor = 252
        elif period == 'weekly':
            factor = 52
        elif period == 'monthly':
            factor = 12
        else:
            raise ValueError("Invalid period. Use 'daily', 'weekly', or 'monthly'.")
        
        volatility = df.std() * (factor ** 0.5)
        print(f"Annualized volatility: {volatility}")
        return volatility

    # Sharpe ratio
    @staticmethod
    def sharpe_ratio(df: pd.Series, riskfree_rate: float = 0.03):
        excess_return = df.mean() - riskfree_rate
        volatility = df.std()
        sharpe_ratio = excess_return / volatility if volatility != 0 else np.nan
        print(f"Sharpe ratio: {sharpe_ratio}")
        return sharpe_ratio

    # Drawdown calculation
    @staticmethod
    def drawdown(return_series: pd.Series, initial_wealth: float = 100):
        prior_peaks_series = return_series.cummax()
        drawdown_series = (return_series - prior_peaks_series) / prior_peaks_series
        return pd.DataFrame({
            "Wealth Index": return_series,
            "Prior Peaks": prior_peaks_series,
            "Drawdown": drawdown_series
        })
    
    # Max drawdown
    @staticmethod
    def max_drawdown(df: pd.Series, initial_wealth):
        drawdown_df = strategyTester.drawdown(df, initial_wealth)
        max_drawdown = drawdown_df["Drawdown"].min()
        print(f"Max drawdown: {max_drawdown} with initial wealth: {initial_wealth}")
        return max_drawdown


In [32]:
# long momentum strategy test
long_momentum_strategy_test = strategyTester(long_return_df, 'monthly', 0.03, 1000)

Annualized return: symbol
BRETT/USDT     59.487767
PEPE/USDT     194.482709
Name: 2024-11-30 00:00:00+00:00, dtype: float64
Annualized volatility: 1.0479081249529167
Sharpe ratio: 3.851762831286035
Max drawdown: 0.0 with initial wealth: 1000


In [33]:
# short momentum strategy test
short_momentum_strategy_test = strategyTester(short_return_df, 'monthly', 0.03, 1000)

Annualized return: symbol
DEGEN/USDT    584.419192
MFER/USDT     126.895147
Name: 2024-11-30 00:00:00+00:00, dtype: float64
Annualized volatility: 1.5865104853641434
Sharpe ratio: 3.3591903812465302
Max drawdown: -0.34227530548324253 with initial wealth: 1000


In [30]:
# Comparison of monthly Long and Short momentum strategies
# Initialize an empty figure
fig = go.Figure()

# Add the first strategy (e.g., long strategy)
fig.add_trace(go.Bar(x=long_return_df.index, y=long_return_df.values, marker_color='blue', name="Long"))

# Add the second strategy (e.g., short strategy)
fig.add_trace(go.Bar(x=short_return_df.index, y=short_return_df.values, marker_color='red', name="Short"))

# Update layout for title and axis labels
fig.update_layout(
    title="Comparison of monthly Long and Short momentum strategies",
    width=1400,
    height=500,
    xaxis_title="Symbol",
    yaxis_title="Cumulative Returns",
    legend_title="Strategies",
)

# Show the plot
fig.show()

In [34]:
# Annualized Return and Max Drawdown from Long and Short Strategies
# Initialize an empty figure
fig = go.Figure()

# Add traces for Long strategy
fig.add_trace(go.Bar(
    x=["Annualized Return (Long)", "Max Drawdown (Long)"], 
    y=[long_momentum_strategy_test.annualized_return_value.mean(), long_momentum_strategy_test.max_drawdown_value], 
    marker_color='blue', 
    name="Long"
))

# Add traces for Short strategy
fig.add_trace(go.Bar(
    x=["Annualized Return (Short)", "Max Drawdown (Short)"], 
    y=[short_momentum_strategy_test.annualized_return_value.mean(), short_momentum_strategy_test.max_drawdown_value], 
    marker_color='red', 
    name="Short"
))

# Update layout
fig.update_layout(
    title="Annualized Return and Max Drawdown from Long and Short Strategies",
    xaxis_title="Metrics",
    yaxis_title="Values",
    barmode='group',  # Group bars by category
    width=800,
    height=400
)

# Show the plot
fig.show()



In [35]:
# Annualized Volatility and Max Drawdown from Long and Short Strategies
# Create a figure
fig = go.Figure()

# Add bars for the Long Strategy - Volatility and Drawdown
fig.add_trace(go.Bar(
    x=["Annualized Volatility(long)", "Sharpe Ratio(long)"], 
    y=[long_momentum_strategy_test.annualized_volatility_value, long_momentum_strategy_test.sharpe_ratio_value],
    name="Long",
    marker_color='blue'
))

# Add bars for the Short Strategy - Volatility and Drawdown
fig.add_trace(go.Bar(
    x=["Annualized Volatility(short)", "Sharpe Ratio(short)"], 
    y=[short_momentum_strategy_test.annualized_volatility_value, short_momentum_strategy_test.sharpe_ratio_value],
    name="Short",
    marker_color='red'
))

# Update layout for grouped bars
fig.update_layout(
    title="Annualized Volatility and Max Drawdown from Long and Short Strategies",
    xaxis_title="Metrics",
    yaxis_title="Metric Value",
    barmode='group',  # Group bars side-by-side
    width=800,
    height=400
)

# Show the plot
fig.show()
